# Estructuras de Datos - Universidad Nacional de Tres de Febrero 
# Trabajo Práctico Nº 2 - Indexador de Noticias

### Lottero Bruno - Leg. 18434

### Dependencias
- configparser
- requests
- beautifulsoup4
- lxml

Se pueden instalar con: pip3 install -r requirements

### A continuación se describen las clases y decisiones de diseño más importantes

## NewsReader.py
#### Esta clase contiene los métodos para leer y almacenar los artículos
#### Contiene métodos para:
- Crear los directorios/archivos xml destino si no existen
- Recolectar y almacenar los artículos según los medios/canales indicados en la configuración

### Decisiones de diseño:
- Para evitar almacenar noticias en un canal, se verifica por xpath que no exista previamente en el mismo archivo XML
- Se utilizó el paquete requests para hacer las peticiones HTTP correspondientes

### Por ejemplo
Leemos la configuración para tests

In [1]:
import configparser

config = configparser.ConfigParser()
config.read('tests/config.ini', encoding='utf-8')

['tests/config.ini']

También definimos una función callback para que nos imprima mensajes por consola (más adelante se explica en más detalle)

In [2]:
def print_callback(message, *args):
    """
    Función callback para recibir mensajes e imprimir un texto adecuado por consola
    """
    messages = {
        "DLERR": "No se pudo descargar el XML de %s",
        "PARSEERR": "No se pudo parsear el XML de %s",
        "NEWARTICLE": "Agregado %s - %s - %s",
        "BADFORMAT": "Mal formato de título o fecha, salteando...",
        "WAITING": "Esperando %s segundos para reiniciar descargas",
        "CANINTERR": "Si lo desea presione CTRL+C para cancelar y volver al menú principal",
        "BLKNE": "El archivo intermedio %s no existe, salteando...",
        "INDERR": "No se puede indexar el artículo con título %s",
        "XMLNF": "No existe el archivo XML %s",
        "MERGEOK": "Construcción del índice invertido finalizada"
    }
    print(messages.get(message) % args)

Construímos NewsReader

In [3]:
from NewsReader import NewsReader

news_reader = NewsReader(config, callback=print_callback)
news_reader.collect_news()

Esperando 1 segundos para reiniciar descargas
Si lo desea presione CTRL+C para cancelar y volver al menú principal
Esperando 1 segundos para reiniciar descargas
Si lo desea presione CTRL+C para cancelar y volver al menú principal


Luego de esto deberíamos tener en tests/TELAM/ los xml de economía y política

## Index.py
#### Es la clase que representa al índice invertido
#### Contiene métodos para:
- Procesar los cuerpos de noticias como bloques (cada medio es un bloque)
- Combinar los índices intermedios en un solo índice

### Decisiones de diseño:
- El formato de los índices intermedios consiste en palabras de longitud variable:
    - 4 bytes de term_id + 4 bytes para indicar tamaño de doc_list + n bytes de doc_list
- El formato de la lista de apariciones final consiste solamente en concatenar cada uno de los doc_list de los intermedios, ya que la metadada se encuentra en el diccionario almacenado con pickle
- En todos los casos las palabras son big endian
- Para hacer el merge en lineas generales:
 - Se coloca un manejador de archivo en una lista por cada bloque intermedio
 - Se hace push de la primer palabra de cada archivo en un min heap
     - Mientras el heap no esté vacío
         - min = Hacer pop del heap
         - next = Leer siguiente palabra a min (para facilitar, min tiene una referencia al manejador de archivo)
         - Si next no está vacío, pushear a heap
         - Si min.term_id coincide con el anterior, acumular los resultados, sinó escribir en disco
- Se utilizó PyStemmer como stemmer
- Para interpretar algunas etiquetas XML se utilizó BeautifulSoup4

In [4]:
from Index import Index

index = Index(config, callback=print_callback)
index.process_blocks()
index.merge_blocks()

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: './tests/TELAM/TELAM.ii.part'

Luego de eso, deberíamos tener en tests/ tres archivos .dict (pickle) y un index.ii (lista de documentos)

## Search.py
#### Esta clase contiene los métodos para hacer búsquedas
#### Contiene métodos para:
- Buscar en el índice invertido con o sin wildcards

### Decisiones de diseño:
- Para buscar una palabra sin wildcard, se decidió que la mejor forma es buscar de forma literal por la raíz usando el stemmer
- Se utilizó el paquete OOBtree para los árboles B

In [6]:
from Search import Search

search = Search(config["DEFAULT"]["output"])
print(search.search_in_ii(["algo", "man*"]))

ModuleNotFoundError: No module named 'Stemmer'